In [2]:
import rioxarray
import xarray as xr
import numpy as np

import os
import glob
import pathlib
import json
import boto3
import s3fs
import shutil
from typing import Any, Dict, Generic, Iterable, List, Optional, TypeVar, Union, cast
from datetime import datetime
from pprint import pprint

import geopandas as gpd
from shapely.geometry import box, mapping, Polygon, LinearRing
import warnings
warnings.filterwarnings("ignore")

import pystac
from pystac import Catalog, get_stac_version, Collection, Item, STACObject, CatalogType, Link
from pystac.stac_io import DefaultStacIO, StacIO
from pystac.extensions.projection import AssetProjectionExtension, ProjectionExtension

In [3]:
# Delete this block once we are in GDAL >= 3.5

awsdir = pathlib.Path("~/.aws").expanduser()
awsdir.mkdir(exist_ok=True)
awscred = awsdir / "credentials"

session = boto3.session.Session()
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()

#Credentials are refreshable, so accessing your access key / secret key
#separately can lead to a race condition. Use this to get an actual matched
#set. 

outstring = f"""
[default]
aws_access_key_id={credentials.access_key}
aws_secret_access_key={credentials.secret_key}
aws_session_token={credentials.token}
"""
with awscred.open(mode='w') as f:
    f.write(outstring)

In [4]:
Bucket = "dh-shift-curated"

In [14]:
pystac.get_stac_version()

'1.0.0'

In [51]:
# Create root STAC catalog
catalog_url = "efs/SBG-SHIFT-STAC/catalog.json"

catalog = pystac.Catalog(id="SBG-SHIFT-STAC",
                         description="STAC catalog for the SBG-SHIFT AVIRIS-NG campaign",
                         title="SBG SHIFT STAC Catalog",
                         href = catalog_url,
                         catalog_type = CatalogType.SELF_CONTAINED
                        )

In [6]:
# Create collection for AVIRIS-NG
# should include AVIRIS-NG-”raw”, (Eventually) AVIRIS-NG-mosaic, Distance to roads, NAIP imagery, DEM
dt1 = datetime(2022, 2, 28, 0) # first flight date
dt2 = datetime(2022, 5, 31, 0) # last day of study
collection_interval = sorted([dt1, dt2])

temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])
spatial_extent = pystac.SpatialExtent([[705900,  3802100, 825900, 3868100]]) # covers all SHIFT fields
extent = pystac.Extent(spatial_extent, temporal_extent)

collection_url = "/efs/SBG-SHIFT-STAC/collection.json"

collection = pystac.Collection(id='AVIRIS-NG',
                               description='SBG-SHIFT AVIRIS_NG data',
                               href=collection_url,
                               license = 'MIT',
                               extent=extent,
                               stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
                              )

collection.providers = [
    pystac.Provider(name="AVIRIS-NG", roles=[pystac.ProviderRole.PRODUCER], url="https://avirisng.jpl.nasa.gov/aviris-ng.html"),
    pystac.Provider(name="SBG", roles=[pystac.ProviderRole.HOST], url="https://sbg.jpl.nasa.gov")
]

In [7]:
# Read in data from S3 bucket
def get_zarrs(Bucket, dataset_date):
    s3 = boto3.client('s3')
    Prefix = f'aviris/{dataset_date}'
    kwargs = {'Bucket': Bucket, 'Prefix': Prefix}
    substring = '100-100-100.zarr'
    links = []
    while True:
        objects = s3.list_objects_v2(**kwargs)
        for obj in objects['Contents']:
            if substring in obj['Key']:
                key = obj['Key']
                url = key[:key.index(substring)+ len(substring)]
                zarr = url.replace(f"aviris/{dataset_date}/", "")
                links.append(str(zarr))
            
        try:
            kwargs['ContinuationToken'] = objects['NextContinuationToken']
        except KeyError:
            break
     
    data_set = set(links)
    data = list(data_set)
    #print(data) # see results
    print("Data retrieved")
    return data

In [28]:
def create_items_add_assets(Bucket, dataset_date, zarr):
    Prefix = f"aviris/{dataset_date}/{zarr}"
    s3_key = os.path.join(Bucket, Prefix)
    
    # Open flight path zarr
    s3 = s3fs.S3FileSystem(anon=False, client_kwargs=dict(region_name='us-west-2'))
    s3_path = f"s3://{s3_key}"
    store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
    ds = xr.open_zarr(store=store, decode_coords="all", consolidated=True)
    print("Zarr read done!")
    
    # Set dimensions and coordinates
    ds = ds.rio.set_spatial_dims(x_dim='x', y_dim='y')
    ds = ds.set_coords(('Easting', 'Northing'))

     # Calculate extent, bbox, and footprint using Easting & Northing
    ul = ds.isel(x=0, y=0)  # Upper left corner
    ur = ds.isel(x=-1, y=0) # Upper right corner
    ll = ds.isel(x=0, y=-1) # Lower left corner
    lr = ds.isel(x=-1, y=-1) # Lower right corner

    ul2 = (np.min(ul.Easting.values), np.max(ul.Northing.values))
    ur2 = (np.max(ur.Easting.values), np.max(ur.Northing.values))
    ll2 = (np.min(ll.Easting.values), np.min(ll.Northing.values))
    lr2 = (np.max(lr.Easting.values), np.min(lr.Northing.values))
    
    extent = Polygon([ul2, ll2, lr2, ur2])
    footprint = mapping(extent)

    # save flight outline as GeoJSON and upload to s3
    substring = '_100-100-100.zarr'
    item_name = zarr[:zarr.index(substring)]

    gpd.GeoDataFrame({"geometry": extent}, index=[0]).to_file(f"{item_name}_flight_outline.geojson", driver='GeoJSON')

    s3 = boto3.client('s3')
    kwargs = {'Bucket': Bucket, 'Key': f"aviris/{dataset_date}/{item_name}_flight_outline.geojson"}
    s3.put_object(**kwargs)
    
    bbox = extent.bounds
    
    # create datetime object from date string
    dt = datetime.strptime(dataset_date, f"%Y%m%d")
    
    item_url = "/efs/SBG-SHIFT-STAC/AVIRIS-NG/{item_name}.json"

    # item should be flight line
    item = pystac.Item(id=f"{item_name}",
                    geometry=footprint, 
                    bbox=bbox, 
                    datetime=dt,
                    href=item_url,
                    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
                    properties={},
                    collection = 'AVIRIS-NG'
                      )
    # add instrument metadata
    item.common_metadata.instruments = ['AVIRIS-NG']

    # add projection extension to item
    proj_ext = ProjectionExtension.ext(item, add_if_missing = True)
    proj_ext.epsg = 32610
    print("Item created!")
    
    # Add dataset asset
    print("Adding assets")
    item.add_asset(
        key="dataset",
        asset=pystac.Asset(
            href= f"https://dh-shift-curated.s3.us-west-2.amazonaws.com/aviris/{dataset_date}/{zarr}"
        )
    )

    # extend the asset with projection extension
    asset_ext = AssetProjectionExtension.ext(item.assets["dataset"])
    asset_ext.epsg = 32610
    asset_ext.bbox = bbox
    asset_ext.geometry = footprint

    # Add flight outline GeoJSON asset
    item.add_asset(
        key="Flight Outline",
        asset=pystac.Asset(
            href= f"https://dh-shift-curated.s3.us-west-2.amazonaws.com/aviris/{dataset_date}/{item_name}_flight_outline.geojson",
            media_type=pystac.MediaType.GEOJSON
            )
    )
    print("Assets added!")
    zarrs_to_items[item_name] = item

In [29]:
# additional dates after 20220412 to be added later
dates = ['20220224', '20220228', '20220308', '20220316', 
        '20220322', '20220405', '20220412'#, '20220420', '20220429', '20220503'
        ]

zarrs_to_items = {}

for dataset_date in dates:
    data = get_zarrs(Bucket, dataset_date)
    
    for zarr in data:
        substring = '_100-100-100.zarr'
        item_name = zarr[:zarr.index(substring)]
        print(f"Starting {item_name}")
        
        # create STAC item and add assets, including Zarr dataset
        create_items_add_assets(Bucket, dataset_date, zarr)

Data retrieved
Starting ang20220224t200332
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t214423
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t213004
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t223027
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t215759
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t195402
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t221356
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t211618
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t204803
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t203333
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t210144
Zarr read done!
Item created!
Adding assets
Assets added!
Starting ang20220224t201928
Zarr read d

In [32]:
# add items to collection
collection.add_items(zarrs_to_items.values())
collection.describe()

Items added to collection!
* <Collection id=AVIRIS-NG>
  * <Item id=ang20220224t200332>
  * <Item id=ang20220224t214423>
  * <Item id=ang20220224t213004>
  * <Item id=ang20220224t223027>
  * <Item id=ang20220224t215759>
  * <Item id=ang20220224t195402>
  * <Item id=ang20220224t221356>
  * <Item id=ang20220224t211618>
  * <Item id=ang20220224t204803>
  * <Item id=ang20220224t203333>
  * <Item id=ang20220224t210144>
  * <Item id=ang20220224t201928>
  * <Item id=ang20220228t195958>
  * <Item id=ang20220228t183924>
  * <Item id=ang20220228t185150>
  * <Item id=ang20220228t204228>
  * <Item id=ang20220228t193333>
  * <Item id=ang20220228t202944>
  * <Item id=ang20220228t205624>
  * <Item id=ang20220228t185720>
  * <Item id=ang20220228t190702>
  * <Item id=ang20220228t210940>
  * <Item id=ang20220228t192104>
  * <Item id=ang20220228t201833>
  * <Item id=ang20220228t194708>
  * <Item id=ang20220308t201508>
  * <Item id=ang20220308t185140>
  * <Item id=ang20220308t192816>
  * <Item id=ang20220

In [53]:
# Add STAC Collection to STAC catalog
catalog.add_child(collection)
catalog.describe()

* <Catalog id=SBG-SHIFT-STAC>
    * <Collection id=AVIRIS-NG>
      * <Item id=ang20220224t200332>
      * <Item id=ang20220224t214423>
      * <Item id=ang20220224t213004>
      * <Item id=ang20220224t223027>
      * <Item id=ang20220224t215759>
      * <Item id=ang20220224t195402>
      * <Item id=ang20220224t221356>
      * <Item id=ang20220224t211618>
      * <Item id=ang20220224t204803>
      * <Item id=ang20220224t203333>
      * <Item id=ang20220224t210144>
      * <Item id=ang20220224t201928>
      * <Item id=ang20220228t195958>
      * <Item id=ang20220228t183924>
      * <Item id=ang20220228t185150>
      * <Item id=ang20220228t204228>
      * <Item id=ang20220228t193333>
      * <Item id=ang20220228t202944>
      * <Item id=ang20220228t205624>
      * <Item id=ang20220228t185720>
      * <Item id=ang20220228t190702>
      * <Item id=ang20220228t210940>
      * <Item id=ang20220228t192104>
      * <Item id=ang20220228t201833>
      * <Item id=ang20220228t194708>
      * <Item

In [54]:
collection.validate()

['https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json',
 'https://stac-extensions.github.io/projection/v1.0.0/schema.json']

In [55]:
catalog.validate()

['https://schemas.stacspec.org/v1.0.0/catalog-spec/json-schema/catalog.json']

In [56]:
# Save STAC Catalog to efs
catalog.normalize_hrefs(catalog_url)
catalog.save(catalog_type=CatalogType.SELF_CONTAINED)
print('Done!')

Done!
